# Curso: Clustering Básico: k-means, DBSCAN e mean shift

## Objetivos:
* Conheça o básico sobre análise exploratória
* Crie visualização de dados utilizando a biblioteca plotly
* Utilize o método K-means, DBSCAN e Mean shift para agrupar dados sem classificação
* Aprenda a agrupar vinhos
* Avalie a qualidade de uma clusterização através da utilização do coeficiente de silhueta
* Parametrize métodos de clusterização através do máximo coeficiente de silhueta

link: https://cursos.alura.com.br/course/clustering-dados-sem-classificacao

## Importando as bibliotécas necessárias

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go